In [4]:
import random
from sqlalchemy.orm import Session
from datetime import datetime, timedelta
from typing import Dict, Any, List
from pydantic import BaseModel, ValidationError
from google import genai
from google.genai import types
import json
import time
import os
import random
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio

load_dotenv()



class Theme(BaseModel):
    title: str
    tone: str
    full_prompt: str
    keyword: str

class ThemeGenerate(BaseModel):
    themes: Theme

def generate_theme_title_and_story(campaign_title: str, insight: str, description: str, target_customer:str) -> List[tuple[str, str]]:
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    
    # Generate response using Gemini API (synchronous version)
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=f"Tạo 5 thương hiệu cho pages với các thông tin {insight} {target_customer}. Mỗi thương hiệu phải có title và keyword. Phải viết bằng tiếng việt, đầu ra ví dụ title: xin chao, keyword: #chuối, #táo, #cam",
        config=types.GenerateContentConfig(
            response_mime_type='application/json',
            response_schema=ThemeGenerate,
            system_instruction=types.Part.from_text(text='hãy biến đổi thông tin đầu vào thành promt hoàn chỉnh theo cấu trúc gồm có title, tone, full_promt dựa trên {description}, ví dụ đầu ra sẽ như sau title: chiến dịch, tone: gần gũi, dễ thương, full_promt: bài viết với giọng văn gần gũi dễ thương dành cho trang facebook nội dung 1000 từ, keyword: #gymer #protein'),
        ),
    )
    
    # Extract the response
    print("Generated 5 themes based on user prompt.")
    content = json.loads(response.text)
    
    # Validate and parse the response using Pydantic
    themes_data = ThemeGenerate(**content)
    
    # Convert list of themes to list of tuples using dot notation for Pydantic model attributes
    return themes_data

In [6]:
a = generate_theme_title_and_story(campaign_title="xin chao", \
    insight="123", description="chiến dịch tạo sự đồng cảm với thương hiệu chuối chín", target_customer="123")
a.model_dump()

Generated 5 themes based on user prompt.


{'themes': {'title': 'Sức Mạnh Việt',
  'tone': 'mạnh mẽ, truyền cảm hứng',
  'full_prompt': 'Bài viết 1000 từ về hành trình chinh phục bản thân, vượt qua giới hạn thể chất và tinh thần của người Việt, với giọng văn mạnh mẽ và truyền cảm hứng. Tập trung vào sự kiên trì, ý chí và tinh thần đồng đội.',
  'keyword': '#SứcMạnhViệt #VuotQuaGioiHan #KienTri'}}